# Duration Prediction

In [47]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import root_mean_squared_error
import pickle
import mlflow
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope
import xgboost as xgb

In [48]:
pd.options.mode.copy_on_write = True

## Preprocessing

In [49]:
def preprocessing(df):
    # compute target variables: trip duration in minutes
    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    # keep only trips of less than an hour
    df = df.loc[(df.duration >= 1) & (df.duration <= 60), :]

    # extract trip date components
    df['trip_month'] = pd.Categorical(df.lpep_pickup_datetime.dt.month)
    df['trip_dom'] = pd.Categorical(df.lpep_pickup_datetime.dt.day)
    df['trip_hour'] = pd.Categorical(df.lpep_pickup_datetime.dt.hour)

    # concatenate pickup and dropoff locations
    df['PU_DO'] = df['PULocationID'].astype(str) + '_' + df['DOLocationID'].astype(str)

    return df

In [50]:
def one_hot_encoding(df, numerical, categorical, dv=None):
    df[categorical] = df[categorical].astype(str)
    df_dicts = df[categorical + numerical].to_dict(orient='records')

    if not dv:
        dv = DictVectorizer()
        dv.fit(df_dicts)

    X_train = dv.transform(df_dicts)

    return X_train, dv

In [51]:
def train_model(model, X_train, y_train):
    model.fit(X_train, y_train)

In [52]:
# read im the raw data
df_train = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet')
df_val = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet')

In [53]:
# data cleaning
df_train = preprocessing(df_train)
df_val = preprocessing(df_val)

In [54]:
# variable selection
categorical = ['PU_DO']
numerical = ['trip_distance']

In [55]:
# categorical feature encoding
X_train, dv = one_hot_encoding(df_train, numerical, categorical)
X_val, _ = one_hot_encoding(df_val, numerical, categorical, dv=dv)

In [56]:
target = 'duration'
y_train = df_train[target]
y_val = df_val[target]

## Experiment Tracking

In [57]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='/Users/bastienwinant/Desktop/projects/mlops-zoomcamp/02-experiment-tracking/mlruns/1', creation_time=1748510026560, experiment_id='1', last_update_time=1748510026560, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [73]:
mlflow.autolog(disable=True)

### Manual Logging
#### Lasso

In [74]:
alpha=.01

In [78]:
lasso = Lasso(alpha=alpha)
train_model(lasso, X_train, y_train)
y_pred = lasso.predict(X_val)

In [79]:
rmse = root_mean_squared_error(y_val, y_pred)

In [81]:
with mlflow.start_run():
    mlflow.set_tag("developer", "Armand Winant")

    # training params
    mlflow.log_param("training-data", 'https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet')
    mlflow.log_param("testing-data", 'https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet')
    mlflow.log_param("alpha", alpha)

    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path='models/lin_reg.bin', artifact_path='models_pickle')

#### XGBoost

In [63]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [64]:
def objective(params):
    booster = xgb.train(
        params=params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, "validation")],
        early_stopping_rounds=50,
        verbose_eval=20
    )

    y_pred = booster.predict(valid)
    rmse = root_mean_squared_error(y_val, y_pred)

    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [65]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:squarederror',
    'seed': 42
}

In [66]:
best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

[0]	validation-rmse:9.42343                           
[20]	validation-rmse:6.62868                          
[40]	validation-rmse:6.58653                          
[60]	validation-rmse:6.55354                          
[80]	validation-rmse:6.53139                          
[100]	validation-rmse:6.51547                         
[120]	validation-rmse:6.49540                         
[140]	validation-rmse:6.48393                         
[160]	validation-rmse:6.47159                         
[180]	validation-rmse:6.46268                         
[200]	validation-rmse:6.45585                         
[220]	validation-rmse:6.45043                         
[240]	validation-rmse:6.44276                         
[260]	validation-rmse:6.43355                         
[280]	validation-rmse:6.42815                         
[300]	validation-rmse:6.42265                         
[320]	validation-rmse:6.41895                         
[340]	validation-rmse:6.41426                         
[360]	vali

### Autologging

In [67]:
mlflow.xgboost.autolog()

2025/05/29 14:40:18 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2025/05/29 14:40:18 INFO mlflow.tracking.fluent: Autologging successfully enabled for xgboost.


In [69]:
params = {
    'learning_rate': 0.19030171678228142,
    'max_depth': 29,
    'min_child_weight': 1.000763936993607,
    'objective': 'reg:squarederror',
    'reg_alpha': 0.20924104314941339,
    'reg_lambda': 0.0025628242268120804,
    'seed': 42
}

In [70]:
booster = xgb.train(
    params=params,
    dtrain=train,
    num_boost_round=1000,
    evals=[(valid, "validation")],
    early_stopping_rounds=50,
    verbose_eval=20
)

y_pred = booster.predict(valid)
rmse = root_mean_squared_error(y_val, y_pred)

2025/05/29 14:44:27 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '0c84192da11947cd8fa2aa106817399c', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow


[0]	validation-rmse:10.71397
[20]	validation-rmse:6.53625
[40]	validation-rmse:6.49724
[60]	validation-rmse:6.47088
[80]	validation-rmse:6.44681
[100]	validation-rmse:6.43238
[120]	validation-rmse:6.41863
[140]	validation-rmse:6.40743
[160]	validation-rmse:6.39713
[180]	validation-rmse:6.38836
[200]	validation-rmse:6.37959
[220]	validation-rmse:6.37267
[240]	validation-rmse:6.36569
[260]	validation-rmse:6.36134
[280]	validation-rmse:6.35735
[300]	validation-rmse:6.35217
[320]	validation-rmse:6.34867
[340]	validation-rmse:6.34376
[360]	validation-rmse:6.33996
[380]	validation-rmse:6.33685
[400]	validation-rmse:6.33479
[420]	validation-rmse:6.33094
[440]	validation-rmse:6.32919
[460]	validation-rmse:6.32751
[480]	validation-rmse:6.32493
[500]	validation-rmse:6.32294
[520]	validation-rmse:6.31874
[540]	validation-rmse:6.31647
[560]	validation-rmse:6.31506
[580]	validation-rmse:6.31557
[600]	validation-rmse:6.31512
[620]	validation-rmse:6.31329
[640]	validation-rmse:6.31306
[660]	validatio

2025/05/29 14:46:35 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.
2025/05/29 14:46:35 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/bastienwinant/Desktop/projects/mlops-zoomcamp/.venv/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [14:46:35] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."
2025/05/29 14:46:55 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
